# Day 15: Chiton

https://adventofcode.com/2021/day/15

## Part 1

In [1]:
import numpy as np

In [2]:
example_txt = """1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581"""

In [3]:
with open('input.txt') as input_file:
    input_txt = input_file.read()

In [4]:
def get_array(txt):
    """Return array of risk levels, the origin and the destination."""
    array = np.array([[int(position) for position in line] for line in txt.strip().split('\n')])
    origin = (0, 0)
    destination = (array.shape[0] - 1, array.shape[1] - 1)
    return array, origin, destination

First try a simple brute-force approach by calculating the total risk of all valid paths, re-using a method used for exploring the cave system for [Day 12](https://adventofcode.com/2021/day/12).

In [5]:
def get_valid_paths(origin, destination):
    """Get a list of all valid paths from the origin to the destination,
    assuming the submarine only moves right or down."""
    valid_paths = []
    queue = [[origin]]
    while queue:
        path = queue.pop()
        (i, j) = path[-1]
        if (i, j) == destination:
            valid_paths.append(path)
        else:
            options = [path + [neighbour] for neighbour in ((i+1, j), (i,j+1)) \
                       if neighbour[0] <= destination[0] and neighbour[1] <= destination[1]]
            queue.extend(options)
    return valid_paths

In [6]:
def get_lowest_total_risk(valid_paths, array, origin):
    """Evaluate the lowest total risk of all valid paths."""
    total_risk = []
    for path in valid_paths:
        risk = 0
        for position in path:
            if position != origin:
                i, j = position
                risk += array[i, j]
        total_risk.append(risk)
    return min(total_risk)

In [7]:
%%time
array, origin, destination = get_array(example_txt)
valid_paths = get_valid_paths(origin, destination)
lowest_total_risk = get_lowest_total_risk(valid_paths, array, origin)
print(f'Lowest total risk is {lowest_total_risk}.')

Lowest total risk is 40.
CPU times: user 583 ms, sys: 22.4 ms, total: 605 ms
Wall time: 256 ms


The solution above works well enough for the 10⨉10 example, but it does not scale for the 100⨉100 puzzle input.

Apparently, a more sophisticated solution is provided by using the [A* search algorithm](https://en.wikipedia.org/wiki/A*_search_algorithm).  The two functions below are mostly copied from the [pseudocode](https://en.wikipedia.org/wiki/A*_search_algorithm#Pseudocode) of the Wikipedia article, with minimal changes to convert from pseudocode to Python and to encode the specifics of the puzzle. 

In [8]:
def reconstruct_path(cameFrom, current):
    """Reconstruct the path once the goal has been reached."""
    total_path = [current]
    while current in cameFrom.keys():
        current = cameFrom[current]
        total_path.insert(0, current)
    return total_path

In [9]:
def A_Star(start, goal, h, array):
    """A* finds a path from start to goal.
    h is the heuristic function. h(n, goal) estimates the cost to reach goal from node n."""
    
    # The set of discovered nodes that may need to be (re-)expanded.
    # Initially, only the start node is known.
    # This is usually implemented as a min-heap or priority queue rather than a hash-set.
    openSet = [start]

    # For node n, cameFrom[n] is the node immediately preceding it on the cheapest path from start
    # to n currently known.
    cameFrom = {}  # an empty map

    # For node n, gScore[n] is the cost of the cheapest path from start to n currently known.
    gScore = {}  # map with default value of Infinity
    gScore[start] = 0

    # For node n, fScore[n] = gScore[n] + h(n). fScore[n] represents our current best guess as to
    # how short a path from start to finish can be if it goes through n.
    fScore = {}  # map with default value of Infinity
    fScore[start] = h(start, goal)

    while openSet:
        # This operation can occur in O(1) time if openSet is a min-heap or a priority queue
        current = min(openSet, key=fScore.get)  # the node in openSet having the lowest fScore[] value
        
        if current == goal:
            return reconstruct_path(cameFrom, current)

        # Construct a list of neighbors of the current node.
        neighbors = []
        if current[0]-1 >= start[0]:
            neighbors.append((current[0]-1, current[1]))
        if current[0]+1 <= goal[0]:
            neighbors.append((current[0]+1, current[1]))
        if current[1]-1 >= start[1]:
            neighbors.append((current[0], current[1]-1))
        if current[1]+1 <= goal[1]:
            neighbors.append((current[0], current[1]+1))

        openSet.remove(current)
        for neighbor in neighbors:
            # d(current,neighbor) is the weight of the edge from current to neighbor
            # tentative_gScore is the distance from start to the neighbor through current
            # tentative_gScore := gScore[current] + d(current, neighbor)
            tentative_gScore = gScore.get(current, float('inf')) + array[neighbor[0], neighbor[1]]
            if tentative_gScore < gScore.get(neighbor, float('inf')):
                # This path to neighbor is better than any previous one. Record it!
                cameFrom[neighbor] = current
                gScore[neighbor] = tentative_gScore
                fScore[neighbor] = tentative_gScore + h(neighbor, goal)
                if neighbor not in openSet:
                    openSet.append(neighbor)

    # Open set is empty but goal was never reached
    print('Failure')
    return []

Define a heuristic function using [taxicab geometry](https://en.wikipedia.org/wiki/Taxicab_geometry) to estimate the cheapest distance to the goal assuming that each position has risk level of 1.

In [10]:
def heuristic(node, goal):
    """Taxicab distance from a given node to the goal."""
    return (goal[0] - node[0]) + (goal[1] - node[1])

In [11]:
def calculate_risk(total_path, array, origin):
    """Calculate the risk from the total path."""
    risk = 0
    for position in total_path:
        if position != origin:
            i, j = position
            risk += array[i, j]
    return risk

Calculate the lowest total risk for the example then the input text.

In [12]:
%%time
for txt in (example_txt, input_txt):
    array, origin, destination = get_array(txt)
    total_path = A_Star(origin, destination, heuristic, array)
    lowest_total_risk = calculate_risk(total_path, array, origin)
    print(f'Lowest total risk is {lowest_total_risk}.\n')

Lowest total risk is 40.

Lowest total risk is 398.

CPU times: user 180 ms, sys: 1.09 ms, total: 181 ms
Wall time: 180 ms


## Part 2

The A* search algorithm also works for an enlarged 500⨉500 array in a reasonable time without modifications.

In [13]:
def get_big_array(array):
    """Enlarge the cave by 5 times in each direction with increased risk levels in each tile."""
    big_array = np.tile(array, (5,5))
    for i in range(big_array.shape[0]):
        for j in range(big_array.shape[1]):
            increased_risk = big_array[i,j] + i//array.shape[0] + j//array.shape[1]
            big_array[i,j] = increased_risk % 10 + increased_risk // 10
    destination = (big_array.shape[0] - 1, big_array.shape[1] - 1)
    return big_array, destination

In [14]:
%%time
for txt in (example_txt, input_txt):
    array, origin, destination = get_array(txt)
    big_array, destination = get_big_array(array)
    total_path = A_Star(origin, destination, heuristic, big_array)
    lowest_total_risk = calculate_risk(total_path, big_array, origin)
    print(f'Lowest total risk is {lowest_total_risk}.\n')

Lowest total risk is 315.

Lowest total risk is 2817.

CPU times: user 13.1 s, sys: 13.6 ms, total: 13.2 s
Wall time: 13.2 s
